# **Comparativo Data-INMET-NASA**


**Paraíba**
https://www.sciencedirect.com/science/article/pii/S0960148123010972?casa_token=LPT3_PeJJU4AAAAA:IcXIBInVyw6JKZkY-qY-jH5dktSKOrRWwCGwb5Po-OVRg1pMc7fe7BJJN1Q3sk-xN5QTq44kQQ 

**Fuentes:**

* https://portal.inmet.gov.br/dadoshistoricos 

* https://power.larc.nasa.gov/data-access-viewer/

**INMET** mide la radiación solar total como __RADIAÇÃO GLOBAL__ en unidades de kJ/m².


**NASA POWER** mide la radiación solar total como __All Sky Surface Shortwave Downward Irradiance" en kWh/m²/día.__

**DATA** del 2029-2025 (28 febrero), dado que a la fecha de hoy 17 de marzo, solo se contaba con el registro en el INMET a esa fecha.


## **Librerías**

In [ ]:
import os
import warnings
import pandas as pd
import seaborn as sns
import requests
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import plotly.graph_objects as go
from IPython.display import display
from scipy.stats import ks_2samp
# from TSPackages import *
from scipy.stats import jarque_bera
from scipy.stats import ks_2samp
from scipy.stats import kurtosis, skew
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
warnings.filterwarnings('ignore')
from pandas.plotting import autocorrelation_plot

## **Data-NASA**

In [ ]:
file_path = r"C:\Users\Keyla Alba\OneDrive - Universidad del Norte\Doctorado (Ciencias)\Move La America\EDA\Datos\POWER_Point_Hourly_20190101_20250228_019d00S_051d00W_LST.csv"
with open(file_path, "r", encoding="utf-8") as file:
    lines = file.readlines()
start_line = next(i for i, line in enumerate(lines) if "YEAR" in line)
df_NASA = pd.read_csv(file_path, skiprows=start_line, encoding="utf-8")
df_NASA['datetime'] = pd.to_datetime({
    'year': df_NASA['YEAR'],
    'month': df_NASA['MO'],
    'day': df_NASA['DY'],
    'hour': df_NASA['HR']
})
print(df_NASA.shape)

## **Data-INMET**

In [ ]:
file_path = [
    r"C:\Users\Keyla Alba\OneDrive - Universidad del Norte\Doctorado (Ciencias)\Move La America\EDA\Datos\INMET_CO_MS_A710_PARANAIBA_01-01-2019_A_31-12-2019.CSV",
    r"C:\Users\Keyla Alba\OneDrive - Universidad del Norte\Doctorado (Ciencias)\Move La America\EDA\Datos\INMET_CO_MS_A710_PARANAIBA_01-01-2020_A_31-12-2020.CSV",
    r"C:\Users\Keyla Alba\OneDrive - Universidad del Norte\Doctorado (Ciencias)\Move La America\EDA\Datos\INMET_CO_MS_A710_PARANAIBA_01-01-2021_A_31-12-2021.CSV",
    r"C:\Users\Keyla Alba\OneDrive - Universidad del Norte\Doctorado (Ciencias)\Move La America\EDA\Datos\INMET_CO_MS_A710_PARANAIBA_01-01-2022_A_31-12-2022.CSV",
    r"C:\Users\Keyla Alba\OneDrive - Universidad del Norte\Doctorado (Ciencias)\Move La America\EDA\Datos\INMET_CO_MS_A710_PARANAIBA_01-01-2023_A_31-12-2023.CSV",
    r"C:\Users\Keyla Alba\OneDrive - Universidad del Norte\Doctorado (Ciencias)\Move La America\EDA\Datos\INMET_CO_MS_A710_PARANAIBA_01-01-2024_A_31-12-2024.CSV",
    r"C:\Users\Keyla Alba\OneDrive - Universidad del Norte\Doctorado (Ciencias)\Move La America\EDA\Datos\INMET_CO_MS_A710_PARANAIBA_01-01-2025_A_28-02-2025.CSV"
]

def load_and_clean(file_path):
    df = pd.read_csv(file_path, sep=';', encoding='latin1', skiprows=8)
    df.columns = [col.strip().upper() for col in df.columns]
    for col in df.columns:
        if "RADIACAO" in col and "KJ" in col:
            df.rename(columns={col: "RADIACAO_GLOBAL"}, inplace=True)
            break
    df['HORA UTC'] = df['HORA UTC'].astype(str).str.replace(' UTC', '', regex=False)
    df['HORA UTC'] = df['HORA UTC'].str.zfill(4)
    df['HORA UTC'] = df['HORA UTC'].str[:2] + ':' + df['HORA UTC'].str[2:4]
    df['datetime'] = pd.to_datetime(df['DATA'] + ' ' + df['HORA UTC'], format='%Y/%m/%d %H:%M', errors='coerce')
    df['RADIACAO_GLOBAL'] = pd.to_numeric(df['RADIACAO_GLOBAL'], errors='coerce').fillna(0)
    return df[['datetime', 'RADIACAO_GLOBAL']]
df_INMET = pd.concat([load_and_clean(fp) for fp in file_path], ignore_index=True)
print(df_INMET.shape)

## **Comportamiento Radiación Solar INMET**

In [ ]:
fig_INMET = px.line(
    df_INMET,
    x='datetime',
    y='RADIACAO_GLOBAL',
    title='Radiación Global (KJ/m²) por Hora INMET (2019 a Febrero 2025)',
    labels={
        'datetime': 'Fecha y Hora',
        'RADIACAO_GLOBAL': 'Radiación (KJ/m²)'
    }
)
fig_INMET.update_layout(template='plotly_white', width=1100, height=500)
fig_INMET.show()

## **Comportamiento Radiación Solar NASA**

In [ ]:
fig_nasa = px.line(
    df_NASA,
    x='datetime',
    y='ALLSKY_SFC_SW_DWN',
    title='Radiación Horaria NASA (2019-2025)',
    labels={'datetime': 'Fecha y hora', 'ALLSKY_SFC_SW_DWN': 'Radiación (W/m²)'}
)
fig_nasa.update_layout(template='plotly_white', width=1100, height=500)
fig_nasa.show()